按照`\tutorials\02-intermediate\recurrent_neural_network`做好前序准备工作

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

创建双向LSTM网络

In [2]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # 初始化状态
        # 由于双向LSTM状态量也是双份的，所以这里比起LSTM的RNN网络有2倍的系数
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # 前向计算，输出(batch_size, seq_length, hidden_size*2)的张量
        out, _ = self.lstm(x, (h0, c0))
        
        # 取最后一个节点做全连接
        out = self.fc(out[:, -1, :])
        return out

模型的训练与测试

In [3]:
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)


# 优化器与loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# 测试模型
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# 保存
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/2], Step [100/600], Loss: 0.5446
Epoch [1/2], Step [200/600], Loss: 0.2442
Epoch [1/2], Step [300/600], Loss: 0.1301
Epoch [1/2], Step [400/600], Loss: 0.1239
Epoch [1/2], Step [500/600], Loss: 0.0929
Epoch [1/2], Step [600/600], Loss: 0.1220
Epoch [2/2], Step [100/600], Loss: 0.1050
Epoch [2/2], Step [200/600], Loss: 0.2362
Epoch [2/2], Step [300/600], Loss: 0.0770
Epoch [2/2], Step [400/600], Loss: 0.0234
Epoch [2/2], Step [500/600], Loss: 0.1901
Epoch [2/2], Step [600/600], Loss: 0.0281
Test Accuracy of the model on the 10000 test images: 98.21 %
